In [56]:
import pandas as pd
# 1. Load trips and delays
file = pd.ExcelFile("Test Data.xlsx")
trips  = pd.read_excel(file, "Trips")
delays = pd.read_excel(file, "Delays")

# 2. Standardize by removing spaces to underscore
trips.columns  = trips.columns.str.strip().str.replace(" ", "_")
delays.columns = delays.columns.str.strip().str.replace(" ", "_")

# 3. Clean Trips, Delays table
#   Drop missing rows
trips = trips.dropna(subset=["Trip", "Date"])
delays = delays.dropna()

#   Validate datetime for the format
trips["Date"] = pd.to_datetime(trips["Date"], errors="coerce")
delays["Date"] = pd.to_datetime(delays["Date"], errors="coerce")

#  using unique key TripKey combining tripdate and trip
trips["TripKey"] = trips["Date"].dt.strftime("%Y-%m-%d") + "-" + trips["Trip"].astype(str)
delays["TripKey"] = delays["Date"].dt.strftime("%Y-%m-%d") + "-" + delays["Trip"].astype(str)

#delays["is_delayed"] = delays["Arrival_Delay"] >= 5       

In [57]:
# Calculating arrival delay
arrival_delay = (
    delays.sort_values(["TripKey", "Event", "Location"])
    .groupby("TripKey")
    .tail(1)[["TripKey", "Arrival_Delay","Location"]]
    .rename(columns={"Arrival_Delay": "FinalArrivalDelay"})
)

In [58]:
arrival_delay

,TripKey,FinalArrivalDelay,Location
0,2011-01-01-747,0,Regina
1,2011-01-01-749,4,Calgary
2,2011-01-01-949,1,Quebec City
3,2011-01-01-950,2,Regina
4,2011-01-01-956,0,Windsor
...,...,...,...
8104,2011-12-31-1247,3,Edmonton
8105,2011-12-31-1253,0,Vancouver
8106,2011-12-31-1259,0,Ottawa–Gatineau
8107,2011-12-31-1802,4,Windsor


In [59]:
#sum the totals of all delays on a trip by Departure Delay and if tiebreaker min of event
aggregate= (
       delays.groupby(['TripKey','Delay_Code','Location'],as_index=False)
             .agg({"Departure_Delay": "sum", "Event": "min"})
            .rename(columns={"Departure_Delay": "TotalDepartureDelay", "Event": "FirstEvent"})
)

In [60]:
# Primary delay code
primary_delay_code =(
       aggregate.sort_values(["TripKey", "TotalDepartureDelay", "FirstEvent"], ascending=[True, False, True])
                     .groupby("TripKey")
                     .head(1)[["TripKey", "Delay_Code"]]
                     .rename(columns={"Delay_Code": "PrimaryDelayCode"})
)

In [61]:
# Setting trip flag when delay greater than 5; so it counts towards OTP Erosion
trip_flags = arrival_delay.copy()
trip_flags["IsDelayed"] = (trip_flags["FinalArrivalDelay"] >= 5).astype(int)

In [62]:
# Final cleaned dataset
cleaned = (
    trips[["TripKey", "Trip", "Date"]]
    .drop_duplicates()
    .merge(trip_flags, on="TripKey", how="left")
    .merge(primary_delay_code, on="TripKey", how="left")
)


In [64]:
cleaned.head(20)

,TripKey,Trip,Date,FinalArrivalDelay,Location,IsDelayed,PrimaryDelayCode
0,2011-01-01-732,732,2011-01-01,NaN,NaN,NaN,NaN
1,2011-01-01-733,733,2011-01-01,NaN,NaN,NaN,NaN
2,2011-01-01-734,734,2011-01-01,NaN,NaN,NaN,NaN
3,2011-01-01-735,735,2011-01-01,NaN,NaN,NaN,NaN
4,2011-01-01-737,737,2011-01-01,NaN,NaN,NaN,NaN
5,2011-01-01-738,738,2011-01-01,NaN,NaN,NaN,NaN
6,2011-01-01-739,739,2011-01-01,NaN,NaN,NaN,NaN
7,2011-01-01-740,740,2011-01-01,NaN,NaN,NaN,NaN
8,2011-01-01-741,741,2011-01-01,NaN,NaN,NaN,NaN
9,2011-01-01-743,743,2011-01-01,NaN,NaN,NaN,NaN


In [ ]:
# Clean, remove all na as zero or None
cleaned["FinalArrivalDelay"] = cleaned["FinalArrivalDelay"].fillna(0)
cleaned["IsDelayed"] = cleaned["IsDelayed"].fillna(0).astype(int)
cleaned["PrimaryDelayCode"] = cleaned["PrimaryDelayCode"].fillna("None")
cleaned["Location"] = cleaned["Location"].fillna("None")

In [ ]:
# Final Cleaned dataset after preprocess used as I/P for PowerBi
cleaned_final = "OTP_Erosion_Cleaned.xlsx"
cleaned.to_excel(cleaned_final, sheet_name="CleanedTrips", index=False)

In [ ]:
cleaned.head(20)